# Webscraping B3 - Coleta da composição dos índices teóricos

<https://www.b3.com.br/pt_br/market-data-e-indices/indices/indices-amplos/indice-ibovespa-ibovespa-composicao-da-carteira.htm/>

In [1]:
import pandas as pd
import requests
import time
import os

from datetime import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By

In [10]:
class INDEX:
    IBOVESPA = 'ibov'
    IBRX_100 = 'ibxx'
    SMALL_CAPS = 'smll'
    DIVIDENDOS = 'idiv'
    FINANCEIRO = 'ifnc'
    INDUSTRIAL = 'indx'
    MATERIAIS_BASICOS = 'imat'
    UTILIDADE_PUBLICA = 'util'
    CONSUMO = 'icon'
    ENERGIA_ELETRIA = 'ieex'
    IMOBILIARIO = 'imob'
    FIIS = 'ifix'
    MIDLARGE_CAPS = 'mlcx'
    COMMODITIES = 'icb'
    BDRS_GLOBAL = 'bdrx'
    SP500 = 'sp500'

# Configura o Selenium, para baixar os dados do IBOV

In [3]:
def get_downloaded_file_path(index_name, custom_download_path=None):
    """
    A função tem como objetivo buscar o diretório padrão de Downloads, a fim
    de retornar o caminho do arquivo baixado via Selenium. Caso o diretório
    padrão tenha sido alterado, é necessário passar o endereço no parâmetro 
    da função.


    Exemplo:
        read_downloaded_file(custom_path='D:\\Downloads')
    """
    # Obter o diretório padrão do usuário atual
    user_home_dir = os.path.expanduser('~')

    if custom_download_path == None:
        # Diretório padrão de downloads para diferentes sistemas operacionais
        if os.name == 'nt':  # Windows
            default_download_dir = os.path.join(user_home_dir, 'Downloads')
        elif os.name == 'posix':  # Unix/Linux/Mac
            default_download_dir = os.path.join(user_home_dir, 'Downloads')
        else:
            default_download_dir = None  # Se não for Windows ou Unix/Linux/Mac
    else:
        default_download_dir = custom_download_path

    if datetime.now().day < 10:
        index_day = f'0{datetime.now().day}'
    else:
        index_day = datetime.now().day

    if datetime.now().month < 10:
        index_month = f'0{datetime.now().month}'
    else:
        index_month = datetime.now().month
        
    file_name = f'\\{str.upper(index_name)}Dia_{index_day}-{index_month}-{datetime.now().strftime("%y")}.csv'
    file_path = default_download_dir + file_name

    return file_path, default_download_dir, file_name

In [4]:
def read_scraped_file(index, custom_download_path):
    """
    Esta função faz a leitura e o tratamento do CSV baixado do site da B3.
    """

    file = get_downloaded_file_path(index, custom_download_path)[0]
    df = pd.read_csv(file, sep=';', encoding='ISO-8859-1', engine='python')

    df = (
        df
        .reset_index()
        .drop(
            columns=df.columns[-1]
        )
        .iloc[:-2]
        .sort_values(by='Código')
        .set_index('Código')
    )

    return df

In [18]:
def scrap_download_index(index, custom_download_path=None):
    """
    Entra no site da B3 via Selenium e baixa o arquivo CSV com a composição do dia.
    O arquivo será baixado no diretório padrão de download.
    """

    # Verifica se o arquivo a ser baixado já existe
    _, folder, file = get_downloaded_file_path(index, custom_download_path=custom_download_path)
    output_path = os.path.join(folder, file[1:])
    if os.path.exists(output_path):
        df = read_scraped_file(index, custom_download_path)
        return df

    # Configurações do navegador
    options = webdriver.ChromeOptions()
    options.add_argument("--start-maximized")  # Maximiza a janela do navegador

    # Inicializa o driver do Chrome
    driver = webdriver.Chrome(options=options)

    url = f'https://sistemaswebb3-listados.b3.com.br/indexPage/day/{index}?language=pt-br'
    driver.get(url)

    # Espera até que o botão de download esteja disponível
    # download_button = WebDriverWait(driver, 10).until(
    #     EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Download')]"))
    # )

    # Encontra o elemento <a> com o texto "Download"
    download_link = driver.find_element(By.XPATH, "//a[contains(text(), 'Download')]")

    # Clica no botão de download
    download_link.click()    

    # espera um tempo antes de fechar o driver
    time.sleep(5)

    # Fecha o navegador
    driver.quit()

    df = read_scraped_file(index, custom_download_path)
    return df

---

In [6]:
def df_to_list(df, yfinance_ticker):
    ticker_list = df.index.tolist()

    if yfinance_ticker:
        ticker_list = list(map(lambda x: x + '.SA', ticker_list))

    return ticker_list

def get_index_constituents(index: str = INDEX.IBOVESPA, to_list=False, yfinance_ticker=True, custom_download_path=None):
    """
    Função que fara o wabscrap dos dados do índice selecionado

    :params
        index: O índice que se quer buscar a composição
        to_list: Marcar 'True' caso queira que a função retorna uma lista com os  tickers que
                 compõem o índice selecionado
        yfinance_ticker: Caso seja utilizado o YFinance para baixar os dados de cotação,
                         marcar 'True'

    return: Um dataframe com as informações dos tickers do índice selecionado ou uma lista
            somente com os tickers, a depender do parâmtro 'to_list'.
    """
    if str.lower(index) == INDEX.SP500:        
        overall = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")[0]
        ticker_list = overall.Symbol
        ticker_list = ticker_list.tolist()

        if yfinance_ticker:
            ticker_list = list(map(lambda x: x.replace('.', '-'), ticker_list))

        if to_list:
            return ticker_list
        
        return overall
    
    else:
        overall = scrap_download_index(index, custom_download_path)

        if to_list:
            ticker_list = df_to_list(overall, yfinance_ticker)
            return ticker_list
        
        return overall

# Ibovespa

In [21]:
ibov = get_index_constituents(INDEX.IBOVESPA, custom_download_path='D:\\Downloads')
ibov

,Ação,Tipo,Qtde. Teórica,Part. (%)
Código,,,,
ABEV3,AMBEV S/A,ON,4.394.245.879,"2,545"
ALOS3,ALLOS,ON NM,532.616.595,"0,556"
ALPA4,ALPARGATAS,PN N1,166.362.038,"0,061"
ASAI3,ASSAI,ON NM,1.349.217.892,"0,596"
AZUL4,AZUL,PN N2,332.825.777,"0,116"
...,...,...,...,...
VBBR3,VIBRA,ON NM,1.114.613.709,"1,301"
VIVA3,VIVARA S.A.,ON NM,125.912.025,"0,151"
VIVT3,TELEF BRASIL,ON,408.343.528,"0,961"


# IBRX 100

In [22]:
ibrx = get_index_constituents(INDEX.IBRX_100, custom_download_path='D:\\Downloads')
ibrx

,Ação,Tipo,Qtde. Teórica,Part. (%)
Código,,,,
ABEV3,AMBEV S/A,ON,4.394.245.879,"2,394"
ALOS3,ALLOS,ON NM,532.616.595,"0,523"
ALPA4,ALPARGATAS,PN N1,166.362.038,"0,058"
ASAI3,ASSAI,ON NM,1.349.217.892,"0,561"
AURE3,AUREN,ON NM,301.966.319,"0,147"
...,...,...,...,...
VBBR3,VIBRA,ON NM,1.114.613.709,"1,224"
VIVA3,VIVARA S.A.,ON NM,125.912.025,"0,142"
VIVT3,TELEF BRASIL,ON,408.343.528,"0,904"


# Small caps

In [23]:
smll = get_index_constituents(INDEX.SMALL_CAPS, custom_download_path='D:\\Downloads')
smll

,Ação,Tipo,Qtde. Teórica,Part. (%)
Código,,,,
ABCB4,ABC BRASIL,PN N2,72.245.617,"0,596"
AESB3,AES BRASIL,ON NM,317.103.937,"1,273"
AGRO3,BRASILAGRO,ON NM,63.815.593,"0,586"
ALOS3,ALLOS,ON NM,532.616.595,"4,364"
ALPA4,ALPARGATAS,PN N1,166.362.038,"0,481"
...,...,...,...,...
VULC3,VULCABRAS,ON ED NM,95.400.547,"0,599"
VVEO3,VIVEO,ON NM,182.249.113,"0,134"
WIZC3,WIZ CO,ON NM,75.273.443,"0,167"


# Concatenando índices

In [24]:
pd.concat([ibov, ibrx], keys=['IBOV', 'SMLL'], axis=1)

IBOV                                               SMLL                                     
              Ação        Tipo  Qtde. Teórica Part. (%)          Ação        Tipo  Qtde. Teórica Part. (%)
Código                                                                                                    
ABEV3    AMBEV S/A          ON  4.394.245.879     2,545     AMBEV S/A          ON  4.394.245.879     2,394
ALOS3        ALLOS  ON      NM    532.616.595     0,556         ALLOS  ON      NM    532.616.595     0,523
ALPA4   ALPARGATAS  PN      N1    166.362.038     0,061    ALPARGATAS  PN      N1    166.362.038     0,058
ASAI3        ASSAI  ON      NM  1.349.217.892     0,596         ASSAI  ON      NM  1.349.217.892     0,561
AZUL4         AZUL  PN      N2    332.825.777     0,116          AZUL  PN      N2    332.825.777     0,109
...            ...         ...            ...       ...           ...         ...            ...       ...
POMO4          NaN         NaN            NaN       NaN     MARCOPOLO  PN      N2    666.413.239     0,194
PSSA3          NaN         NaN            NaN       NaN  PORTO SEGURO  ON      NM    181.531.496     0,254
SMFT3          NaN         NaN            NaN       NaN     SMART FIT  ON      NM    307.405.718     0,301
STBP3          NaN         NaN            NaN       NaN    SANTOS BRP  ON  EDJ NM    855.677.944     0,499
TEND3          NaN         NaN            NaN       NaN         TENDA  ON      NM    121.681.013     0,071

[97 rows x 8 columns]